## PBL(2): Pest Classification - template code

#### PBL(2) 프로젝트를 수행하기 위한 템플릿 코드입니다. 

1. https://agtechresearch.pythonanywhere.com/ 에 가입된 이메일과 비밀번호를 사용해주세요.

    (회원가입을 하지 않았다면, 위 사이트에 접속하여 회원가입해 주세요: 비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [1]:
project = "pestclassification"  # 수정하지 마세요
username = "abc@abc.co.kr"  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
password = "1234"  # 비밀번호

리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [2]:
import os
import urllib.request

if not os.path.exists("competition.py"):
    url = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/competition/competition.py"
    filename = "competition.py"
    urllib.request.urlretrieve(url, filename)

### 데이터셋 다운로드

아래 코드를 실행하여 데이터(dataset.zip)를 다운로드 받고, dataset 폴더에 압축을 해제합니다.

    - 학습용 데이터셋: trainset
    - problem set: problemset

데이터 다운로드와 압축 해제가 성공적으로 진행되었다면, 2개의 폴더와 2개의 csv 파일이 dataset 폴더 내부에 생성됨

* trainset 폴더: 학습용 데이터셋. 작물재배에 유해한 10종류의 해충 이미지들(aphids, armyworm, blisterbeetle, cicadellidae, cornborer, cricket, delicatula, limacodidae, miridae, viridis)
* problemset 폴더: 학습된 모델에 의하여 예측을 수행하여야 할 데이터셋 -> 리더보드 제출을 위한 problem

* problem.csv: 예측을 수행하여야 할 데이터셋 이미지들의 FilePath 포함
* submission.csv: 리더보드 서버 제출을 위한 파일 형식


In [ ]:
!wget https://github.com/agtechresearch/LectureAlgorithm/raw/main/pestclassification/dataset.zip

!unzip dataset.zip -d dataset

In [3]:
from IPython.display import Image
import pandas as pd
import torch
import warnings

# Data 경로 설정
DATA_DIR = "dataset"

# 경고 무시
warnings.filterwarnings("ignore")

In [4]:
# 데이터 디렉토리 설정: trainset, problemset
train_dir = os.path.join(DATA_DIR, 'trainset')
problem_dir = os.path.join(DATA_DIR, 'problemset')

### 학습용 데이터셋을 폴더로부터 읽어들이기 위해 ImageFolder 사용

 - 폴더에 위치한 이미지를 tensor 형식으로 변환
 - 폴더 내 위치한 이미지에 대하여 라벨링 기능 지원
 
 

In [5]:
Image(url='https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/img/folderstruct.png', width=150)

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 이미지 전처리 정의
train_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),  # 이미지 크기를 (32, 32)로 조정
        transforms.Grayscale(num_output_channels=1),  # 흑백 이미지 load (채널 1개)
        transforms.RandomHorizontalFlip(0.5),  # 50% 확률로 랜덤하게 이미지 좌우 반전
        transforms.RandomRotation(90),  # 랜덤하게 이미지 회전
        transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
        #transforms.Normalize((0.5,),(0.5,)) # 이미지를 정규화
    ]
)

# ImageFolder를 사용하여 학습용 데이터셋 trainset 생성
trainset = datasets.ImageFolder(root=train_dir, transform=train_transform)

# DataLoader 인스턴스 train_loader 생성
# batch_size 의 경우 일반적으로는 16 혹은 32 로 설정
# num_workers 는 대개 컴퓨터의 코어 수와 동일하게 설정
train_loader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)

In [7]:
# trainset의 클래스 정보 확인 (추후 학습된 모델에 의해 예측 결과가 숫자로 도출됨 -> 숫자 라벨을 클래스 이름으로 변환하여 사용하기 위함)
print(trainset.class_to_idx)

# 숫자 라벨을 클래스 이름으로 매핑하는 딕셔너리 생성
idx_to_class = {v: k for k, v in trainset.class_to_idx.items()}

{'aphids': 0, 'armyworm': 1, 'blisterbeetle': 2, 'cicadellidae': 3, 'cornborer': 4, 'cricket': 5, 'delicatula': 6, 'limacodidae': 7, 'miridae': 8, 'viridis': 9}


In [8]:
# shape 확인 (batch_size, channel, height, width)
x, y = next(iter(train_loader))
x.shape, y.shape

(torch.Size([16, 1, 32, 32]), torch.Size([16]))

### 네트워크 모델 정의

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Device 설정 (cuda:0 혹은 cpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 모델 정의
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        # 마지막 출력층의 Neuron은 Class 개수로 설정
        self.output = nn.Linear(32, num_classes)

    def forward(self, x):
        # (B, 1, 32, 32) -> (B, 32*32)
        x = x.view(-1, 32 * 32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.output(x)
        return x

cpu


In [10]:
# 모델 생성
model = Net(num_classes=10)

# 모델을 device 에 올림 (cuda:0 혹은 cpu)
model.to(device)
model

Net(
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=10, bias=True)
)

### 손실함수 및 옵티마이저 정의/설정

In [ ]:
# Cross Entropy Loss 정의
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 설정: model.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=0.005)

### 모델 학습

In [ ]:
# 최대 반복 횟수 정의
num_epoch = 20

# loss 기록하기 위한 list 정의
losses = []
accs = []

# 훈련 모드 활성화
model.train()

for epoch in range(num_epoch):
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0

    for x, y in train_loader:
        # x, y 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        x = x.to(device)
        y = y.to(device)

        # 그라디언트 초기화 (초기화를 수행하지 않으면 계산된 그라디언트는 누적됩니다.)
        optimizer.zero_grad()

        # output 계산: model의 __call__() 함수 호출
        output = model(x)

        # 손실(loss) 계산 (예측 Prediction, 실제 Ground Truth)
        loss = loss_fn(output, y)

        # 미분 계산
        loss.backward()

        # 경사하강법 계산 및 적용
        optimizer.step()

        # 배치별 loss 를 누적합산 합니다.
        running_loss += loss.item()
        running_acc += output.argmax(dim=1).eq(y).sum().item() / len(y)

    # 누적합산된 배치별 loss값을 배치의 개수로 나누어 Epoch당 loss를 산출합니다.
    loss = running_loss / len(train_loader)
    losses.append(loss)
    acc = running_acc / len(train_loader)
    accs.append(acc)

    # 매 Epoch의 학습이 끝날때 훈련 결과를 출력합니다.
    print(f"{epoch:03d} loss = {loss:.5f}, accuracy = {acc:.5f}")

print("Training finished")

### problemset 이미지들을 불러오기

In [ ]:
problem = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))
problem.head()

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]  # 'FilePath' 열에서 이미지 경로 가져오기
        image = Image.open(img_path).convert("L")  # 이미지를 흑백으로 로드

        if self.transform:
            image = self.transform(image)

        return image


# 이미지를 불러올 때 적용할 전처리 정의: resize, to tensor
problem_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),  # 이미지 크기를 (28, 28)로 조정
        transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
    ]
)

# 커스텀 데이터셋 인스턴스 생성
custom_dataset = CustomImageDataset(dataframe=problem, transform=problem_transform)

# DataLoader 인스턴스 생성
problem_loader = DataLoader(custom_dataset, batch_size=1, shuffle=False)

## Problem set 문제에 대한 예측 및 리더보드 결과 제출

In [ ]:
predictions = []

# 검증모드 진입
model.eval()

with torch.no_grad():
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0
    for x in problem_loader:
        x = x.to(device)

        y_hat = model(x)
        label = y_hat.argmax(dim=1).detach().item()
        predictions.append(label)

In [ ]:
# 숫자 라벨을 클래스 이름으로 변환
your_answer = [idx_to_class[l] for l in predictions]

- 아래 제출 프로세스가 느리다고 중지 후 다시 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능: 최대 100회까지 가능)
- 제출에 성공할 경우, "제출에 성공하였습니다"의 메세지와 함께 제출 결과 accuracy 가 화면에 출력됩니다.
- 제출결과는 또한 [대회 페이지(리더보드 서버)](https://agtechresearch.pythonanywhere.com/competitions/pestclassification/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.

In [ ]:
import competition

# 리더보드 서버 제출을 위한 파일 생성(예측 결과 업데이트)
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["Label"] = your_answer

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)